In [2]:
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
import torch
import torch.nn as nn
import random
import json
from torch.utils.data import Dataset, DataLoader
from tkinter import *

In [3]:
with open("707.json", "r") as f:
    intents = json.load(f)

In [4]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

In [5]:
def bag_of_words(tokenized_sentence, words):
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1

    return bag

In [6]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents["intents"]:
    tag = intent["tag"]
    # add to tag list
    tags.append(tag)
    for pattern in intent["patterns"]:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))
print(xy)

[(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['Yo', '!'], 'greeting'), (['Howdy'], 'greeting'), (['Nice', 'to', 'meet', 'you', '.'], 'greeting'), (['Goodbye'], 'Goodbye'), (['Bye', '!'], 'Goodbye'), (['Adios'], 'Goodbye'), (['See', 'ya', 'later', '!'], 'Goodbye'), (['TTYL'], 'Goodbye'), (['Cya'], 'Goodbye'), (['Bye'], 'Goodbye'), (['Good', 'bye'], 'Goodbye'), (['Bye'], 'Goodbye'), (['See', 'you', 'later', '.'], 'Goodbye'), (['Have', 'a', 'great', 'day', '.'], 'Goodbye'), (['See', 'you', 'next', 'time', '.'], 'Goodbye'), (['It', 'was', 'my', 'pleassure', '.'], 'Goodbye'), (['Take', 'care', '.'], 'Goodbye'), (['See', 'ya', '!'], 'Goodbye'), (['Catch', 'you', 'later', '.'], 'Goodbye'), (['Ciao', '.'], 'Goodbye'), (['I', 'should', 'go'], 'Goodbye'), (['What', 'are', 'you', 'doing', '?'], 'activity'), (['Wyd'], 'acti

In [7]:
# stem and lower each word
ignore_words = ["?", ".", "!"]
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [8]:
print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

333 patterns
29 tags: ['Goodbye', 'Saeyoung', 'activity', 'affirmative', 'agreement', 'animals', 'compliments', 'creator', 'emotes', 'excitement', 'favor', 'food', 'gratitude', 'greeting', 'hacker', 'joke', 'learn', 'mean', 'morning', 'name', 'negatives', 'night', 'positive', 'religion', 'romantic', 'sad', 'unsure', 'work', 'worry']
381 unique stemmed words: ["'m", "'re", "'s", "'ve", '(', ')', ',', '...', '3', '707', ':', ':3', '<', '^-^', 'a', 'about', 'account', 'actor', 'actual', 'address', 'adio', 'afford', 'afraid', 'alcohol', 'all', 'alway', 'am', 'an', 'and', 'ani', 'anim', 'anxiou', 'anymor', 'anyon', 'anyth', 'appreci', 'are', 'arm', 'around', 'artifici', 'assist', 'at', 'atheist', 'attract', 'awesom', 'bad', 'be', 'beat', 'beauti', 'becaus', 'bed', 'been', 'belief', 'believ', 'best', 'bet', 'better', 'bike', 'birth', 'bottom', 'bright', 'buddha', 'bug', 'busi', 'but', 'buy', 'by', 'bye', 'c', 'ca', 'call', 'can', 'car', 'care', 'cat', 'catch', 'chat', 'chicken', 'chip', 'cho

In [9]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

In [10]:
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train)
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[13 13 13 13 13 13 13 13 13 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  2  2  2  2  2  2  2  2  2  2  2 19 19 19 19 19 19 19 19 19
 19 19 19 19 19  1  1  1  1  1  1  1 11 11 11 11 11 11 11 11 11 11 23 23
 23 23 23 23 23 23 23 23 23 23 23 24 24 24 24 24 24 24 24 24 24 24 24 15
 15 15 15 15 15 15 15 15 15 15  5  5  5  5  5  5  5  5  5  5  5  5  5 17
 17 17 17 17 17 17 17 17 17 17 17 17 28 28 28 28 28 28 28 28 28 28 28 28
 28 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25  8  8  8  8  8  8  8  8
  8  8 21 21 21 21 21 21 21 21 21 21 21 21 18 18 18 18 18 18 18 18 18 18
  3  3  3  3  3  3  3  3  3  3 20 20 20 20 20 20 20 20 20 20 26 26 26 26
 26 26 26 26 26 26  6  6  6  6  6  6  6  6  6  6  6  6 12 12 12 12 12 12
 12 12 12 12  9  9  9  9  9  9  9  9  9  9 27 27 27 27 27 27 27 27 27 27
 27 27 14 14 14 14 14 14 14 14 14 14  4  

In [11]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [12]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [13]:
# Hyper-parameters
num_epochs = 2000
batch_size = 8
learning_rate = 0.0001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)

In [14]:
dataset = ChatDataset()
train_loader = DataLoader(
    dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0
)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [17]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [18]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [100/2000], Loss: 3.0971
Epoch [200/2000], Loss: 2.5975
Epoch [300/2000], Loss: 1.7940
Epoch [400/2000], Loss: 1.8341
Epoch [500/2000], Loss: 1.0649
Epoch [600/2000], Loss: 0.9052
Epoch [700/2000], Loss: 0.4593
Epoch [800/2000], Loss: 0.6424
Epoch [900/2000], Loss: 0.0904
Epoch [1000/2000], Loss: 0.1624
Epoch [1100/2000], Loss: 0.0643
Epoch [1200/2000], Loss: 0.0833
Epoch [1300/2000], Loss: 0.0452
Epoch [1400/2000], Loss: 0.0154
Epoch [1500/2000], Loss: 0.0049
Epoch [1600/2000], Loss: 0.0131
Epoch [1700/2000], Loss: 0.0036
Epoch [1800/2000], Loss: 0.0037
Epoch [1900/2000], Loss: 0.0039
Epoch [2000/2000], Loss: 0.0036


In [19]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags,
}

In [20]:
FILE = "data.pth"
torch.save(data, FILE)

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
with open("707.json", "r") as json_data:
    intents = json.load(json_data)

In [23]:
FILE = "data.pth"
data = torch.load(FILE)

In [24]:
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

In [25]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=381, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=29, bias=True)
  (relu): ReLU()
)

In [26]:
bot_name = "707"

In [27]:
confused = ["Hmm…", "Hmmmm~.", "Hmm..","Hmm lol","Hmm~.","Hmm.","Hmm;","Hmm;;","Hmm, hmm~","Hmm hmm hmm~",
            "…?","…", "…!","!?", "Hmm…!","Uhm…","Uhm.","Uhm, uh…..","Uhm···","Oh?","Oh!","Oh…", "Oh.", 
            "Oh….","Oh lololol","Oh….lolol","Oh!!!","…Oh.","Oh…!", "Oh ho~", "Huh?","Huh…?",
            "W…","What?", "Ahem~!","Ahem lol","Ahem!","Ahem >.<","I don’t really understand what this means."]

In [28]:
def get_response(msg):
    sentence = tokenize(msg)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                return random.choice(intent["responses"])
    return random.choice(confused)

In [29]:
BG_RED = "#EEC630"
BG_COLOR = "#AD3B20"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"

In [30]:
your_name = input("What's your name: ")

What's your name: Ari


In [ ]:
class ChatApplication:
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()

    def run(self):
        self.window.mainloop()

    def _setup_main_window(self):
        self.window.title("Chat")
        self.window.resizable(width=False, height=False)
        self.window.configure(width=470, height=550, bg=BG_COLOR)

        # head label
        head_label = Label(
            self.window,
            bg=BG_COLOR,
            fg=TEXT_COLOR,
            text="Welcome",
            font=FONT_BOLD,
            pady=10,
        )
        head_label.place(relwidth=1)

        # tiny divider
        line = Label(self.window, width=450, bg=BG_RED)
        line.place(relwidth=1, rely=0.07, relheight=0.012)

        # text widget
        self.text_widget = Text(
            self.window,
            width=20,
            height=2,
            bg=BG_COLOR,
            fg=TEXT_COLOR,
            font=FONT,
            padx=5,
            pady=5,
        )
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)

        # scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)

        # bottom label
        bottom_label = Label(self.window, bg=BG_RED, height=80)
        bottom_label.place(relwidth=1, rely=0.825)

        # message entry box
        self.msg_entry = Entry(bottom_label, bg=BG_COLOR, fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)

        # send button
        send_button = Button(
            bottom_label,
            text="Send",
            font=FONT_BOLD,
            width=20,
            bg=BG_RED,
            command=lambda: self._on_enter_pressed(None),
        )
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)

    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, your_name)

    def _insert_message(self, msg, sender):
        if not msg:
            return

        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)

        msg2 = f"{bot_name}: {get_response(msg)}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)

        self.text_widget.see(END)


if __name__ == "__main__":
    app = ChatApplication()
    app.run()